# Acquire ACS Data Over Serial WITHOUT Knowledge From the Device File
For this example, you will need to connect an ACS to your computer via serial. 
The sensor should be on before running this notebook. Note that the ACS is in picoDOS for the first 10 seconds after it is powered on and may not register on a serial port.

An example for working with ACS data WITH knowledge of the device file can be found [here](https://github.com/IanTBlack/acspype/tree/main/examples).
Collecting data from the ACS without a device file is not generally recommended, but if you are in a pinch and the device file is unknown, this is an example of how to do it.

Note: In this example, absorption and attenuation will look bad because the ACS is running in air.

In [1]:
from datetime import datetime, timezone  # Used for comparing timestamps in the gap test.

from acspype import ACSStream # A convenience class for reading and parsing ACS packets over serial.
from acspype.processing import convert_sn_str # Processing and conversion functions for ACS data.
import acspype.qaqc as acsqaqc  # QAQC functions for ACS data.
from acspype.utils import find_acs_port # This is a convenience function that finds the first port that has an ACS attached to it.

In [2]:
num_packets = 5  # Acquire X packets, and then stop. Otherwise, the while loop will run forever.

In [3]:
port = find_acs_port()  # Find the port that has an ACS attached to it.
with ACSStream(port) as acss: # Open an ACS serial object.
    i = 0
    while True:  # Continuously read the stream until a full packet is found.
        acss.read_stream()
        acs_pkt = acss.find_packet()
        if acs_pkt.full_packet is not None:  # If a full packet is found, then perform operations on it.
            parsed_packet = acss.parse_packet(acs_packet = acs_pkt) # Parse the packet.
            
            # Run the gap and syntax tests.
            flag_gap = acsqaqc.gap_test(now = datetime.now(timezone.utc), 
                                        time_stmp = acs_pkt.daq_time, 
                                        buffer_length= len(acss._buffer), # The bytes in the serial buffer can be accessed with the _buffer attribute. This is probably the only application instance where you would need access to the buffer contents external of the ACSStream class.
                                        record_length = parsed_packet.record_length)
            flag_syntax = acsqaqc.syntax_test(full_packet = acs_pkt.full_packet)
            
            # Print some of the results.
            print(f"---------- Packet {i+1} ----------")
            print(f"Acquisition Time: {parsed_packet.daq_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'}")
            print(f"Acquired Packet From: {convert_sn_str(parsed_packet.sn_int)}")
            print(f"Elapsed Time (ms): {parsed_packet.elapsed_time}")
            print(f"Absorption Signal Counts: {parsed_packet.a_signal}")
            if flag_gap == 1:
                print('Gap Test: Passed')
            elif flag_gap == 4:
                print('Gap Test: Failed')
            if flag_syntax == 1:
                print('Syntax Test: Passed')
            elif flag_syntax == 4:
                print('Syntax Test: Failed')
            print('\n')
            
            i += 1
            if i == num_packets:
                break

---------- Packet 1 ----------
Acquisition Time: 2025-05-07T16:52:18.960Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 600162
Absorption Signal Counts: (461, 530, 616, 721, 843, 980, 1126, 1303, 1504, 1739, 1995, 2238, 2505, 2803, 3143, 3490, 3882, 4284, 4733, 5254, 5798, 6343, 6966, 7576, 8197, 8833, 9428, 10114, 10867, 11581, 12306, 13143, 14015, 15062, 16136, 17156, 18204, 19172, 20257, 21285, 22130, 23083, 23979, 24937, 26066, 26843, 28833, 29708, 30552, 31260, 31968, 32581, 33247, 33901, 34227, 34406, 34526, 34559, 34487, 34332, 33902, 33220, 32454, 31577, 31151, 30820, 29901, 28917, 27943, 26823, 25759, 24691, 23475, 22087, 20719, 19577, 18503, 17327, 16147, 15034, 13932, 12945, 12028, 11091)
Gap Test: Passed
Syntax Test: Passed


---------- Packet 2 ----------
Acquisition Time: 2025-05-07T16:52:19.211Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 600412
Absorption Signal Counts: (462, 532, 617, 723, 847, 979, 1128, 1305, 1504, 1740, 1995, 2239, 2506, 2804, 3145, 3490